# TM10007 Group Assignment Machine Learning
#### Sara Arman, Judith Essenburg, George Franssen, Naomi Verkerk

## Google colab environment

In [ ]:
# Run this to use from colab environment
#!pip install -q --upgrade git+https://github.com/naomiverkerk/TM10007.git

## Import


In [ ]:
from load_data import load_data
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

## Loading Data

In [ ]:
def load_data():
    data = pd.read_csv('TCGA_GBM_radiomicFeatures.csv', na_values=['#DIV/0!'], index_col=0)
    data2 = pd.read_csv('TCGA_LGG_radiomicFeatures.csv', na_values=['#DIV/0!'], index_col=0)
    data['label'] = 'GBM'
    data2['label'] = 'LGG'
    data = data.append(data2)
    data = data.drop(columns=['Date'])
    return data


In [ ]:
data = load_data() 

print(f'The number of samples/patients: {len(data.index)}')
print(f'The number of columns/features: {len(data.columns)}')


In [ ]:
y = data.label
ax = sns.countplot(y,label="Count")     
GBM, LGG = y.value_counts()
print('Number of GBM: ',GBM)
print('Number of LGG: ',LGG)

## Splitting in train and test data

In [ ]:
# the code to split, after that we inspect the data

## Checking/exploring the dataset

In [ ]:
data.dtypes == 'object'
# Seperate numerical variables and categorical variables
num_vars = data.columns[data.dtypes != 'object']
cat_vars = data.columns[data.dtypes == 'object']

print(len(num_vars))
print(len(cat_vars))
print(cat_vars)

In [ ]:
data2 = data.iloc[155:158, 7:9]
data2

In [ ]:
GBM = data[data.label == 'GBM']
LGG = data[data.label == 'LGG']
ax = sns.boxplot(data= orient="h", palette="Set2")




In [ ]:
# to check which features have most missing values
missing_values = data[num_vars].isnull().sum().sort_values(ascending=False)
missing_values_multiple = (missing_values > 0).sum()

print(missing_values)
print(f'The total number of features with 1 or more missing values is {missing_values_multiple}')



In [ ]:
# saving missing values in a variable
a = pd.isna(data).sum()/len(data)*100

# saving column names in a variable
variables = data.columns

# new variable to store variables having missing values less than a threshold

variable = [ ]

for i in range(data.columns.shape[0]):
    if a[i]<=40: #setting the threshold as 40%
        variable.append(variables[i])

variable   

# # creating a new dataframe using the above variables

# new_data = data[variable]
# new_data.head()

# # percentage of missing values in each variable of new data

# p = pd.isna(new_data).sum()/len(new_data)*100

import numpy as np
x = data.iloc[:,:].isin([np.inf, -np.inf]).sum()
x[x>0]

a[a >= 50]

In [ ]:
GBM = data[data.label == 'GBM']
LGG = data[data.label == 'LGG']

In [ ]:
import warnings
warnings.filterwarnings('ignore')

y1 = data.columns
y2 = GBM.columns
y3 = LGG.columns

plt.figure(figsize=(20,20))

for i in range(0,9):
    
    nb = i+1
    plt.subplot(3,3,nb)
    sns.distplot(GBM[y[i]])

In [ ]:
import warnings
warnings.filterwarnings('ignore')

y = LGG.columns

plt.figure(figsize=(10,10))

for i in range(0,20):
    nb = i+1
    plt.subplot(5,5,nb)
    sns.distplot(LGG[y[i]])

In [ ]:
import warnings
warnings.filterwarnings('ignore')
plt.figure(figsize=(10,10))
y = data.columns
for i in range(0,20):
    nb = i+1
    plt.subplot(5,5,nb)
    sns.distplot(data[y[i]])

plt.show()